In [6]:
!mkdir build
%cd build
!cmake ../
!make

mkdir: cannot create directory ‘build’: File exists
/home/aseal/Documents/invint/build
-- Configuring done
-- Generating done
-- Build files have been written to: /home/aseal/Documents/invint/build
[  3%] Built target dtnode
[  5%] Performing update step for 'antlr4_runtime'
Ignoring path runtime/Cpp/runtime/thirdparty/utfcpp/
Current branch master is up to date.
Already up to date!
[  6%] Performing configure step for 'antlr4_runtime'
loading initial cache file /home/aseal/Documents/invint/build/antlr4_runtime/tmp/antlr4_runtime-cache-.cmake
CMake Warning at CMakeLists.txt:7 (message):
  Build type not set, falling back to Release mode.

   To specify build type use:
   -DCMAKE_BUILD_TYPE=<mode> where <mode> is Debug or Release.


-- Building without demo. To enable demo build use: -DWITH_DEMO=True
CMake Deprecation Warning at CMakeLists.txt:31 (CMAKE_POLICY):
  The OLD behavior for policy CMP0054 will be removed from a future version
  of CMake.

  The cmake-policies(7) manual explai

In [7]:
import time
import subprocess
import numpy as np
import json
import pandas as pd

In [8]:
def time1(query):
    q = open("query.txt","w")
    q.write(query + "\n")
    q.close()
    times = []
    out = ""
    for i in range(5):
        d1 = time.process_time()
        try:
            sp = subprocess.run("./main2 --model=dt --file=dt.txt < query.txt", shell = True, capture_output = True, timeout = 10)
            out = sp.stdout.splitlines()[2]
        except:
            print("timeout with query " + query)
            times = [10,10,10,10,10]
            out = "timeout"
            break
        #timeout!!!
        d2 = time.process_time()
        times.append(d2-d1)
    return (times , out)

def random_constant(query , dt_size):
    ans = ""
    idx = 1
    for i in range(len(query[0])):
        if (query[0][i] == 'C'):
            ans+="[ "
            for j in range(dt_size):
                if (j):
                    ans += " , "
                val = query[idx][j]
                if (val == 2): val = '?'
                else: val = str(val)
                ans += val
            ans += " ] "
            idx+=1
        else:
            ans+=query[0][i]
    return ans

def generate_query(squeries , nvars , rng):
    query = ""
    isnot = rng.integers(2)
    if (isnot):
        query += " ~ ( "
    if (squeries == 1):
        
        if (nvars == 1):
            opt = rng.integers(3)
        else:
            opt = rng.integers(4)
        
        if (opt == 0):
            
            query += " P ( x" + str( rng.integers(nvars) + 1 ) + " ) "
        
        if (opt == 1):
            
            query += " C <= x" + str( rng.integers(nvars) + 1 ) + " "
        
        if (opt == 2):
            
            query += " x" + str( rng.integers(nvars) + 1 ) + " <= C "
            
        if (opt == 3):
            
            x1 = rng.integers(nvars) + 1
            x2 = rng.integers(nvars) + 1
            while (x2 == x1):
                x2 = rng.integers(nvars) + 1
            query += " x" + str(x1) + " <= x" + str(x2) + " "
        
    else:
        
        query += " ( "
        fs = rng.integers(squeries-1) + 1
        query += generate_query(fs , nvars , rng)
        
        if ( rng.integers(2) ):
            query += " ) V ( "
        else:
            query += " ) ^ ( "
            
        query += generate_query(squeries - fs , nvars , rng) + " ) "
    
    if (isnot):
        query += " ) "
    
    return query

def time_queries(dt_name , queries, rng):
    
    data = {}
    
    with open("../dtrees/" + dt_name) as json_file:
        data = json.load(json_file)
        
    with open('dt.txt', 'w') as outfile:
        json.dump(data, outfile)
    
    times = []
    timesr = []
    
    for q in queries:
        queryt = random_constant(q , len( data["feature_names"] ) )
        ctimes = time1( queryt )
        timep = ( sum(ctimes[0]) - max(ctimes[0]) - min(ctimes[0]))/3
        timesr.append( timep )
        finalarr = [queryt] + ctimes[0] + [ctimes[1]] + [timep]
        times.append(finalarr)
        
    df = pd.DataFrame( times , columns = ["query" , "t1" , "t2" , "t3" , "t4" , "t5" , "ans" , "tp"] )
    
    df.to_csv(dt_name + ".csv")
    
    return timesr

def test_trees(qsize, nvars , nqueries, seed):
    data = []
    rng = np.random.default_rng(seed)
    queries = []
    dims = [3, 5, 10, 15, 20, 25, 30]
    leaves =  [5, 10, 15, 20, 50, 100, 150, 200, 500, 1000, 2000]
    for i in range(nqueries):
        q = ""
        ex = rng.integers(2)
        for j in range(nvars):
            if (ex):
                q += "Exists x" + str(j + 1) + ", "
            else:
                q += "Forall x" + str(j + 1) + ", "
        q += generate_query(qsize , nvars , rng)
        nconstants = q.count('C')
        fquery = [q]
        for i in range(nconstants):
            c = []
            for j in range( max(dims) ):
                c.append( rng.integers(3) )
            fquery.append(c)
        queries.append(fquery)
    infop = [ ]
    for l in leaves:
        for d in dims:
            infop.append( [d , l ] +  time_queries( "dim_" + str(d) + "_nleaves_" + str(l) + ".json" , queries , rng) )
            print( str(l) + " " + str(d) + " " + str(sum(infop[len(infop)-1][2:])/nqueries) )
    
    qs = []
    for i in range(nqueries):
        qs.append( queries[i][0] )
    df = pd.DataFrame( infop , columns = ["dimension" , "leaves" ] + qs )
    df.to_csv("final_data.csv")

In [ ]:
test_trees(10,3,10,5)

5 3 0.006612696299999913
5 5 0.0065254496333333025
5 10 0.006482110933333373
timeout with query Forall x1, Forall x2, Forall x3,  ~ (  (  ~ (  P ( x1 )  )  ) V (  (  (  (  ~ (  x1 <= [ ? , 0 , 1 , ? , 1 , ? , 0 , 1 , ? , 1 , 1 , 1 , 0 , ? , ? ]   )  ) V (  ~ (  x3 <= [ 0 , 0 , ? , 0 , 1 , 1 , 1 , 1 , 1 , ? , 0 , 1 , ? , ? , 1 ]   )  )  ) V (  (  [ ? , 0 , 1 , 1 , 1 , 1 , ? , ? , 0 , 0 , 1 , 0 , 1 , 1 , 1 ]  <= x3  ) ^ (  (  ~ (  [ ? , 1 , ? , ? , ? , 1 , 0 , 0 , 1 , 1 , 1 , 1 , ? , ? , 1 ]  <= x2  )  ) V (  ~ (  (  x3 <= x1  ) V (  (  x2 <= x3  ) ^ (  ~ (  (  x2 <= x3  ) V (  ~ (  x1 <= [ ? , ? , ? , ? , 1 , 1 , 1 , 0 , 0 , ? , ? , 0 , 1 , 1 , 0 ]   )  )  )  )  )  )  )  )  )  ) ^ (  x1 <= [ 0 , ? , 0 , 1 , ? , 0 , ? , 0 , 0 , 0 , 1 , ? , 0 , ? , 0 ]   )  )  ) 
timeout with query Exists x1, Exists x2, Exists x3,  (  ~ (  (  ~ (  (  (  ~ (  x2 <= x1  )  ) ^ (  ~ (  [ 0 , 1 , ? , ? , 1 , 1 , ? , 0 , 1 , ? , 0 , 0 , ? , 1 , ? ]  <= x3  )  )  ) V (  x2 <= [ 0 , 1 , 0 , ? , ? , 1 , ? , 0 , 0

timeout with query Forall x1, Forall x2, Forall x3,  (  (  (  ~ (  x2 <= [ ? , ? , 0 , 1 , 0 , 1 , 0 , 1 , ? , 1 , ? , 0 , 0 , 1 , ? , ? , 0 , 0 , 0 , 1 , 1 , 1 , 0 , ? , ? , 0 , 1 , 1 , 1 , ? ]   )  ) ^ (  x3 <= [ 1 , ? , 0 , 1 , ? , ? , ? , 1 , 1 , ? , 0 , ? , ? , 1 , ? , ? , 1 , 0 , 1 , 1 , 0 , 1 , 0 , 0 , 1 , 1 , 0 , 0 , 1 , 0 ]   )  ) V (  (  ~ (  (  (  [ 0 , 0 , 0 , ? , ? , 1 , 1 , 1 , 0 , 0 , ? , ? , ? , 1 , ? , ? , ? , ? , 0 , 1 , 1 , 1 , 1 , 1 , 0 , 1 , ? , ? , 1 , 0 ]  <= x1  ) V (  ~ (  [ 1 , 1 , ? , ? , 0 , ? , 0 , 1 , ? , 0 , 0 , ? , 0 , 1 , 0 , 0 , 0 , 1 , 1 , 1 , 0 , 1 , 0 , 1 , 0 , 0 , 0 , 1 , ? , ? ]  <= x2  )  )  ) V (  (  x2 <= x3  ) ^ (  ~ (  (  P ( x2 )  ) V (  ~ (  P ( x3 )  )  )  )  )  )  )  ) ^ (  (  P ( x1 )  ) V (  x1 <= [ ? , 0 , ? , 1 , 1 , 0 , 0 , ? , 1 , 1 , 1 , 1 , 1 , 0 , ? , ? , ? , 1 , ? , ? , 1 , 0 , 0 , 1 , ? , 1 , ? , 0 , 0 , 1 ]   )  )  )  ) V (  ~ (  x1 <= x2  )  ) 
timeout with query Forall x1, Forall x2, Forall x3,  ~ (  (  ~ (  P ( x1 )  )  ) V